# Building A Chatbot

In this notebook, we'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

> **Note:** This chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- **Conversational RAG**: Enable a chatbot experience over an external source of data  
- **Agents**: Build a chatbot that can take actions


In [23]:
## OpenAI API and Groq Key and Opensource models
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [24]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BBCE27DA80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BBCE27E050>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello,My name is Faraz,and I am an AI Full Stack Developer.")])

AIMessage(content='Hello Faraz, it\'s nice to meet you! \n\nThat\'s an impressive title - "AI Full Stack Developer".  \n\nWhat kind of projects are you working on these days? I\'m always interested to hear about the innovative things people are doing with AI.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 26, 'total_tokens': 86, 'completion_time': 0.109090909, 'prompt_time': 0.002142935, 'queue_time': 0.017442685000000003, 'total_time': 0.111233844}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e025d8f-ce99-4990-a33f-0a32a24027a5-0', usage_metadata={'input_tokens': 26, 'output_tokens': 60, 'total_tokens': 86})

In [26]:
## Example of Messages to LLM and whether it remember them-Yes
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hello,My name is Faraz,and I am an AI Full Stack Developer."),
    AIMessage(content="Hello Faraz, it's nice to meet you! That's a very impressive title. \n\nAs an AI, I'm always interested in learning more about how people are using technology, especially in fields like full stack development. \n\nWhat kind of projects are you working on these days?  Do you have a particular area of full stack development that you specialize in?"),
    HumanMessage(content="Hey what's my name and what do I do?")
]

response=model.invoke(messages)

In [27]:
response

AIMessage(content="You are Faraz, and you are an AI Full Stack Developer!  \n\nRemember, as a large language model, I don't have memory of past conversations.  So each time we interact, it's like a fresh start. 😊  \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 126, 'total_tokens': 181, 'completion_time': 0.1, 'prompt_time': 0.005520079, 'queue_time': 0.018112101, 'total_time': 0.105520079}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-64755da5-00c8-4084-b8ed-7a7e73ea13b0-0', usage_metadata={'input_tokens': 126, 'output_tokens': 55, 'total_tokens': 181})

# Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let’s see how to use this!

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [29]:
config = {
    "configurable": {
        "session_id": "user_faraz_001"
    }
}


In [30]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, I am Faraz, an AI Software Engineer.")],
    config=config
)

In [31]:
response.content

"Hello Faraz! It's nice to meet you.\n\nAs an AI myself, I'm always interested in connecting with other people working in the field. What kind of AI software engineering do you specialize in? Are you working on any exciting projects right now?\n"

In [32]:
## Checking if AI remembers my name
with_message_history.invoke(
    [HumanMessage(content="What is my Name?")],
    config=config
)

AIMessage(content='Your name is Faraz, as you introduced yourself at the beginning of our conversation. 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 91, 'total_tokens': 113, 'completion_time': 0.04, 'prompt_time': 0.005295489, 'queue_time': 0.017333838, 'total_time': 0.045295489}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b01d49d-2270-45e4-9714-14e1a610fe00-0', usage_metadata={'input_tokens': 91, 'output_tokens': 22, 'total_tokens': 113})

In [33]:
## Changing config-->session id--> To check if it still remembers
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [34]:
## Re-tell him your name---> now it remembers
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [35]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model


In [37]:
chain.invoke({"messages": [HumanMessage(content="Hi,my name is Muhammad Faraz.")]})

AIMessage(content="Hello Muhammad Faraz, it's nice to meet you!  \n\nWhat can I help you with today?  😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 34, 'total_tokens': 64, 'completion_time': 0.054545455, 'prompt_time': 0.003367684, 'queue_time': 0.018094675, 'total_time': 0.057913139}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-651c3be2-437b-4366-b86c-964528b86895-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64})

In [38]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)


In [40]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Faraz")],
    config=config
)


In [41]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Faraz. 😊  \n\nIs there anything else you'd like to know or talk about?\n"

In [42]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [49]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Faraz")],"language":"Urdu"})
response.content

'اسلام علیکم فرارز! \n\nخوش آمدید. میں آپ کی مدد کرنے کے لیے تیار ہوں. آپ مجھ سے جو بھی پوچھیں، میں اپنی تمام سہولیات کے ساتھ جواب دوں گا۔\n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [50]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [51]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Faraz")],"language":"Turkish"},
    config=config
)
repsonse.content

'Merhaba Faraz, \n\nNasıl yardımcı olabilirim? 😊 \n'

In [52]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

In [53]:
response.content

'Your name is Faraz.  \n\nI remember that from our first interaction. 😊  Is there anything else I can help you with?\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [56]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\faroo\OneDrive\Desktop\LCEL_APP\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\faroo\OneDrive\Desktop\LCEL_APP\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\faroo\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [57]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences! 🍦\n\nWhat's your favorite flavor? 😊\n"

In [60]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what did I ask you of having?")],
        "language": "English",
    }
)
response.content

"You asked if I was having fun! 😊  \n\n\n\nI'm glad you're enjoying our conversation.  Is there anything else I can help you with?\n"

In [61]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [62]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you. So I don't know your name. 😊\n\nWould you like to tell me?\n"

In [63]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"